## Exercise 5: Classification

### 5.1. Parameter optimization
In Exercise 4.2 we have used the German credit data set from the UCI data set library (http://archive.ics.uci.edu/ml/index.html), which describes the customers of a bank with respect to whether they should get a bank credit or not.

#### 5.1.1.	(recap) Go back to the results of exercise 4.2.4. Re-run the classifiers with their default parameter settings.
- Used the 10-fold validation approach.
- Balanced the training set multiplying the “bad customer” examples. 
- Evaluated the results, setting up your cost matrix to ((0,100)(1,0)) – that is, you assumed you will lose 1 Unit if you refuse a credit to a good customer, but that you lose 100 Units if you give a bad customer a credit.

Rerun your process to get the performance results. What were the default parameters of the Decision Tree?

In [1]:
import itertools
import matplotlib.pyplot as plt
import numpy as np

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

First, we load the dataset:

In [4]:
import pandas as pd
from scipy.io import arff
from sklearn.preprocessing import LabelEncoder

credit_arff_data, credit_arff_meta = arff.loadarff(open('credit-g.arff', 'r'))
credit_data = pd.DataFrame(credit_arff_data)

# select all columns of type object
columns_with_binary_strings = credit_data.select_dtypes('object').columns.values

# decode the values of these columns using utf-8
credit_data[columns_with_binary_strings] = credit_data[columns_with_binary_strings].apply(lambda x: x.str.decode("utf-8"))
credit_target = credit_data['class']
credit_data = credit_data.drop(columns='class')

label = LabelEncoder()
credit_target = label.fit_transform(credit_target)
label_names=['bad','good']
label_order=label.transform(label_names)

credit_data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Desktop/ML Group 2/credit-g.arff'

Then, we set up a pipeline and evaluate it using cross validation:

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
#from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import make_scorer

numeric_features = ['duration', 'credit_amount', 'installment_commitment', 'residence_since', 'age', 'existing_credits', 'num_dependents']
categorical_features = ['credit_history', 'purpose', 'personal_status', 'other_parties', 'property_magnitude', 'other_payment_plans', 'housing', 'job', 'own_telephone', 'foreign_worker']
ordinal_features = [ 'checking_status', 'savings_status', 'employment']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features),
        ('ord', OrdinalEncoder(categories=[
            [ 'no checking', '<0', '0<=X<200', '>=200' ],
            [ 'no known savings', '<100', '100<=X<500', '500<=X<1000', '>=1000' ],
            [ 'unemployed', '<1', '1<=X<4', '4<=X<7', '>=7' ]
        ]), ordinal_features)])

pipeline = Pipeline([ 
    ('preprocessing', preprocessor), 
    ('balancing', RandomOverSampler()),
    ('estimator', DecisionTreeClassifier()) ])

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

prediction = cross_val_predict(pipeline, credit_data, credit_target, cv=cv)

def cost_function(y_true, y_pred): 
    cm = confusion_matrix(y_true, y_pred, labels=label_order)
    return cm[0][1] * 100 + cm[1][0] * 1

cm = confusion_matrix(credit_target, prediction, labels=label_order)
cost = cost_function(credit_target, prediction)
acc = accuracy_score(credit_target, prediction)

print("Decision Tree with accuracy of {} and cost {}".format(acc, cost))
plot_confusion_matrix(cm, classes=label_names, title='Decision Tree Classifier')
plt.show()
print(classification_report(credit_target, prediction, target_names=label_names))

Finally, we fit the pipeline to the dataset and plot the decision tree:

In [ ]:
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.utils.multiclass import unique_labels

pipeline.fit(credit_data, credit_target)

estimator = pipeline.named_steps['estimator']
pre = pipeline.named_steps['preprocessing']
feature_names = numeric_features + list(pre.named_transformers_['cat'].get_feature_names_out(categorical_features)) + ordinal_features

plt.figure(figsize=(20,10))
tree.plot_tree(estimator,
               feature_names=feature_names,
                class_names=label_names)
plt.show()

Default Parameters for decision tree:

In [ ]:
DecisionTreeClassifier().get_params()

#### 5.1.2.	Now try to find a more optimal configuration for the Decision Tree. What is the best parameter setting and what is the cross-validated performance of this appraoch?

Use the GridSearchCV from scikit learn. Try the following parameters of the Decision Tree:
- criterion: ['gini', 'entropy']
- 'max_depth': [2, 3, 4, 5, None]
- 'min_samples_split' :[2,3,4,5]

What is the best configuration for the data set and the classification approach? 

Note: The grid search can take some time. You can use the ```n_jobs=-1``` parameter setting for the ```cross_val_predict()``` function to enable parallel processing (all CPU cores will be used).

In [ ]:
from sklearn.model_selection import GridSearchCV

# define the parameter grid
parameters = {
    'estimator__criterion':['gini', 'entropy'], 
    'estimator__max_depth':[ 2, 3, 4, 5, None],
    'estimator__min_samples_split' :[2,3,4,5]
}

# define the folds for the cross validation
stratified_10_fold_cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# create a scorer for the grid search
cost_score = make_scorer(cost_function, greater_is_better=False)

# create the grid search estimator
grid_search_estimator = GridSearchCV(pipeline, parameters, scoring=cost_score, cv=stratified_10_fold_cv)

# cross-validate
prediction = cross_val_predict(grid_search_estimator, credit_data, credit_target, cv=cv, n_jobs=-1)

# calculate costs
cm = confusion_matrix(credit_target, prediction, labels=label_order)
cost = cost_function(credit_target, prediction)
acc = accuracy_score(credit_target, prediction)

print("Optimised Decision Tree with accuracy of {} and cost {}".format(acc, cost))
plot_confusion_matrix(cm, classes=label_names, title='Decision Tree Classifier')
plt.show()
print(classification_report(credit_target, prediction, target_names=label_names))

# fit the grid search (= determine the optimal parameters)
grid_search_estimator.fit(credit_data, credit_target)
print("Optimised Parameters: {}".format(grid_search_estimator.best_params_))

#### 5.1.4.	How does the optimal decision tree differ from the one you have learned in 4.2.4?
Plot the optimised tree!

In [ ]:
estimator = grid_search_estimator.best_estimator_.named_steps['estimator']
pre = grid_search_estimator.best_estimator_.named_steps['preprocessing']
feature_names = numeric_features + list(pre.named_transformers_['cat'].get_feature_names_out(categorical_features)) + ordinal_features

plt.figure(figsize=(20,10))
tree.plot_tree(estimator,
               feature_names=feature_names,
                class_names=label_names)
plt.show()